# 自注意力解决的问题      

## 输入层面
- **input is a vector**      

通常认为输入的内容可以看作一个**向量**   

- **input is a set of vectors**

其中这组**向量的个数可能不同**，每个**向量的长度可能不同**——因此这是自注意力解决的问题

    - 例子   
    1. 文字处理，每个句子长度是不一样的(向量长度不一样)——one hot/embedding等
    2. 语音处理，在语音中划分window,该window内处理成的向量叫做Frame
    3. Graph拓扑图，社交网络中每个节点都有一个向量
    
## 输出层面    

- **每个向量对应一个label**
    - 词性标注的工作，每个词语对应一个label


- **整个序列对应一个label**          
    - sentiment analysis——整个句子对应一个极性
    
- **模型决定应该输出几个label**(seq2seq)
    - 语音辨识    
    - 文本翻译

# Self-Attention模块

## Sequence Labeling中的问题

假设一个词性标注工作，当只考虑单个词，或者一个***window***内的向量可能无法达到很好的效果，需要考虑一整个***sequence***

如果窗口大于输入序列的长度，可以考虑进整个序列的信息，但是这样的全连接可能导致的是过拟合并且计算量非常大    

<img style="float: center;" src="resource_pic/self_attention/self_attention_1.PNG" width=600 height=600>

## Self-attention 

在一个序列输入***self-attention***时，该层输出***vec***数量相同的***sequence***，这个序列每个向量**各自**经过全连接层后，输出对应的***label***。自注意力层可以叠加多次，如下图:

<img style="float: center;" src="resource_pic/self_attention/self_attention_2.PNG" width=500 height=500>


下图的$a^i$代表一个输入序列，或者上一个隐层的输出；$b^i$代表经过了自注意力层的输出。          

$a^i$需要与$a^j(i\neq j)$计算$relevant$(相关性)，作为***attention score***

<img style="float: center;" src="resource_pic/self_attention/self_attention_3.PNG" width=500 height=500>

***Self-attention***包括***dot-product***(点积注意力)以及***additive-attention***(加性注意力)，如下图所示     

其中加性注意力通过将乘以***query,key***矩阵的两个向量**串联**在一起达成

<img style="float: center;" src="resource_pic/self_attention/self_attention_4.PNG" width=500 height=500>


## Dot-Product

***Dot-Product***首先需要计算***attention score***$\alpha_{i,j}$，即第$i$个序列**对**第$j$的**注意力分数**。

1. 首先每个输入序列对应一个***query和key***($q,k$)，两者对应的矩阵为$W_q,W_k$<font color = "orange">(该参数在所有***vec***上共享)</font>，即$q^i = W_qa^i,k^i = W_ka^i$


2. 如图所示，以$a^1$为例，对计算出的所有$k^i$(包括$k^1$)与$q^1$相乘得到$\alpha_{1,j}$，最后通过***softmax***层得到$\alpha'_{1,j}$
<img style="float: center;" src="resource_pic/self_attention/self_attention_5.PNG" width=500 height=500>

3. 每个输入序列对应一个***value***($v$)，对应矩阵为$W_v$(共享参数)，即$v^i = W_va^i$,最后通过如下公式计算$b^1$

$$b^1 = \sum^i\alpha_{1,i}v^i$$

<img style="float: center;" src="resource_pic/self_attention/self_attention_6.PNG" width=500 height=500>

## Dot-Product的并行操作   

自注意力机制可以并行操作导致在计算效率上大大提高:

1.计算***query,key,value***

将$a^i$列成矩阵$I = [a^1,a^2,a^3,a^4]$，则$Q = [q^1,q^2,q^3,q^4] = W^qI$,同理$K = W^kI,V=W^vI$

<img style="float: center;" src="resource_pic/self_attention/self_attention_7.PNG" width=450 height=450>

2. 计算***attention score***       
    - attention score的矩阵$A'$的大小是$L\times L$，其中$L$是sequence的长度，因此在sequence过长时计算量会过大

$$
A'\overset{softmax}{\leftarrow}A = \left[\begin{matrix}
\alpha_{1,1}&\alpha_{2,1}&\alpha_{3,1}&\alpha_{4,1}\\
\alpha_{1,2}&\alpha_{2,2}&\alpha_{3,2}&\alpha_{4,2}\\
\alpha_{1,3}&\alpha_{2,3}&\alpha_{3,3}&\alpha_{4,3}\\
\alpha_{1,4}&\alpha_{2,4}&\alpha_{3,4}&\alpha_{4,4}
\end{matrix} \right ]
=[k^1,k^2,k^3,k^4]^T[q^1,q^2,q^3,q^4]=K^TQ
$$

<img style="float: center;" src="resource_pic/self_attention/self_attention_8.PNG" width=450 height=450>

3. 计算自注意力层输出值    

$$O = [b^1,b^2,b^3,b^4] = VA' = [v^1,v^2,v^3,v^4]
\left[\begin{matrix}
\alpha'_{1,1}&\alpha'_{2,1}&\alpha'_{3,1}&\alpha'_{4,1}\\
\alpha'_{1,2}&\alpha'_{2,2}&\alpha'_{3,2}&\alpha'_{4,2}\\
\alpha'_{1,3}&\alpha'_{2,3}&\alpha'_{3,3}&\alpha'_{4,3}\\
\alpha'_{1,4}&\alpha'_{2,4}&\alpha'_{3,4}&\alpha'_{4,4}
\end{matrix} \right ]$$

<img style="float: center;" src="resource_pic/self_attention/self_attention_9.PNG" width=450 height=450>

## 多头注意力层(Multi-head Self-attention)

<img style="float: left;" src="resource_pic/self_attention/self_attention_10.PNG" width=450 height=450>

<img style="float: left;" src="resource_pic/self_attention/self_attention_11.PNG" width=450 height=450>

## Positional-Encoding

## Truncated Self-Attention

# Self-Attention的应用场景

## CNN与自注意力的关系

## LSTM与自注意力的关系   

## 在Graph上的应用场景